<a href="https://www.kaggle.com/code/shahrish99/alexnet-mnist?scriptVersionId=256937409" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/alexnet_mnist/pytorch/default/1/alexnet_mnist.pth


In [3]:
import numpy as np
import pandas as pd
import os
import torch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using torch device: {device}')

# Data preprocessing and augmentation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert MNIST grayscale to 3 channels
    transforms.Resize((224, 224)),  # Resize to AlexNet's input size
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize between -1 and 1 for compatibility
])

# Load MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root='/data', train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.MNIST(
    root='/data', train=False, download=True, transform=transform
)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load pre-trained AlexNet
model = torchvision.models.alexnet(pretrained=True)

# Modify the classifier for MNIST (10 output classes)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)

# Move model to device
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5  # You can increase this for better performance
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

# Validation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Validation"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

# Save the model
torch.save(model.state_dict(), "alexnet_mnist.pth")
print("Model saved as alexnet_mnist.pth")

Using torch device: cuda


Training Epoch 1/5:   2%|▏         | 23/938 [00:03<02:23,  6.36it/s]


KeyboardInterrupt: 

In [5]:
# Validation loop
model.eval()
top1_correct = 0
top5_correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Validation"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        # Get top-1 and top-5 predictions
        _, top5_preds = outputs.topk(5, dim=1)

        # Top-1 accuracy
        top1_correct += (top5_preds[:, 0] == labels).sum().item()

        # Top-5 accuracy
        top5_correct += sum([labels[i] in top5_preds[i] for i in range(labels.size(0))])

        total += labels.size(0)

# Compute Top-1 and Top-5 accuracies
top1_accuracy = 100 * top1_correct / total
top5_accuracy = 100 * top5_correct / total

print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")


Validation: 100%|██████████| 157/157 [00:22<00:00,  7.05it/s]

Top-1 Accuracy: 98.63%
Top-5 Accuracy: 99.91%


In [6]:
# Import necessary libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Specify the path to the saved checkpoint
checkpoint_path = '/kaggle/input/alexnet_mnist/pytorch/default/1/alexnet_mnist.pth'  # Update with the actual path after uploading

# Load the saved state dictionary
model.load_state_dict(torch.load(checkpoint_path))
print(f"Model loaded from {checkpoint_path}")

/tmp/ipykernel_23/2719223783.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


Model loaded from /kaggle/input/alexnet_mnist/pytorch/default/1/alexnet_mnist.pth


In [7]:
!pip install torchattacks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.1.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.


In [8]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [10]:
import torchattacks

# Ensure model is in eval mode
model.eval()

# -------------------------
# PGD Attack (Projected Gradient Descent)
# -------------------------
pgd = torchattacks.PGD(model, eps=0.01, alpha=2/255, steps=40)
print("\nEvaluating model on PGD adversarial examples...")

pgd_correct = 0
pgd_total = 0

for images, labels in tqdm(test_loader, desc="PGD Attack"):
    images, labels = images.to(device), labels.to(device)
    adv_images = pgd(images, labels)
    outputs = model(adv_images)
    _, predicted = torch.max(outputs.data, 1)
    pgd_total += labels.size(0)
    pgd_correct += (predicted == labels).sum().item()

pgd_acc = 100 * pgd_correct / pgd_total
print(f"PGD Attack Accuracy: {pgd_acc:.2f}%")


Evaluating model on PGD adversarial examples...


PGD Attack: 100%|██████████| 157/157 [04:40<00:00,  1.79s/it]

PGD Attack Accuracy: 91.01%


In [13]:
# -------------------------
# CW Attack (Carlini & Wagner)
# -------------------------
cw = torchattacks.CW(model, c=1e-3, kappa=0, steps=100, lr=0.01)
print("\nEvaluating model on CW adversarial examples...")

cw_correct = 0
cw_total = 0

for images, labels in tqdm(test_loader, desc="CW Attack"):
    images, labels = images.to(device), labels.to(device)
    adv_images = cw(images, labels)
    outputs = model(adv_images)
    _, predicted = torch.max(outputs.data, 1)
    cw_total += labels.size(0)
    cw_correct += (predicted == labels).sum().item()

cw_acc = 100 * cw_correct / cw_total
print(f"CW Attack Accuracy: {cw_acc:.2f}%")


Evaluating model on CW adversarial examples...


CW Attack: 100%|██████████| 157/157 [02:01<00:00,  1.29it/s]

CW Attack Accuracy: 94.99%


In [ ]:
import torchattacks
# List of attacks with their parameters
attacks = {
    "FGSM": torchattacks.FGSM(model, eps=0.3),
    "PGD": torchattacks.PGD(model, eps=0.3, alpha=2/255, steps=40),
    "CW": torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)
}

# Evaluate model on each attack
for attack_name, attack in attacks.items():
    correct = 0
    total = 0
    print(f"Running {attack_name} Attack...")
    
    for inputs, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        adv_inputs = attack(inputs, labels)  # Generate adversarial examples
        outputs = model(adv_inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f"{attack_name} Adversarial Accuracy: {100 * correct / total:.2f}%\n")

In [ ]:
import torchattacks
# PGD Attack
pgd_attack = torchattacks.PGD(model, eps=0.3, alpha=2/255, steps=40)

# Function to test adversarial accuracy
def adversarial_test(attack, loader, attack_name):
    model.eval()
    correct = 0
    total = 0
    
    for inputs, labels in tqdm(loader, desc=f"Adversarial Test ({attack_name})"):
        inputs, labels = inputs.to(device), labels.to(device)
        adv_inputs = attack(inputs, labels)
        outputs = model(adv_inputs)
        
        _, pred = outputs.max(1)
        correct += pred.eq(labels).sum().item()
        total += labels.size(0)
    
    acc = correct / total
    print(f'{attack_name} Accuracy: {acc:.4f}')
    return acc

# Evaluate model with PGD attack
pgd_acc = adversarial_test(pgd_attack, test_loader, "PGD")

In [ ]:
torch.